# Serving a Stable Diffusion Model¶

This example runs a Stable Diffusion application with Ray Serve.

This example uses the [stabilityai/stable-diffusion-2](https://huggingface.co/stabilityai/stable-diffusion-2) model and [FastAPI](https://fastapi.tiangolo.com/) to build the example

Steps:
1. Execute the stable_diffusion_on_ray.ipynb notebook to initiate the ray server and import the model
2. Run the below inference steps

### Define Ray Service

In [ ]:
ray_cluster_headnode_svc = "http://example-cluster-kuberay-head-svc:8000"

### Inference using single prompt

In [ ]:
import requests
from IPython.display import Image, display

prompt = "An helicopter fying over snow capped mountains"
# prompt = "A serious capybara at work, wearing a suit"
# prompt = "A Squirtle fine dining with a view to the London Eye"
# prompt = "A tamale food cart in front of a Japanese Castle"
# prompt = "a graffiti of a robot serving meals to people"
# prompt = "a beautiful cabin in Attersee, Austria, 3d animation style"

input = "%20".join(prompt.split(" "))
resp = requests.get(f"{ray_cluster_headnode_svc}/imagine?prompt={input}")

display(Image(resp.content))

### Inference for Multiple Prompts

In [ ]:
import requests
import multiprocessing
import random
from io import BytesIO
from PIL import Image

def send_request_and_receive_image(prompt: str):
  """Sends a single prompt request and returns the Image."""
  inputs = "%20".join(prompt.split(" "))
  resp = requests.get(f"{ray_cluster_headnode_svc}/imagine?prompt={inputs}")

  return BytesIO(resp.content)


def send_requests():
  """Sends a list of requests and processes the responses."""
  prompts = [
      "Labrador in the style of Hokusai",
      "Painting of a squirrel skating in New York",
      "HAL-9000 in the style of Van Gogh",
      "Times Square under water, with fish and a dolphin swimming around",
      "A rocket race car driving around the milky way",
      "A house carried by balloons across the rocky mountains",
      "Armchair in the shape of an avocado",
      "Clown astronaut in space, with Earth in the background",
  ]
  with multiprocessing.Pool(processes=len(prompts)) as p:
    raw_images = p.map(send_request_and_receive_image, prompts)

  images = [Image.open(raw_image) for raw_image in raw_images]

  def image_grid(imgs, rows, cols):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
      grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

  grid = image_grid(images, 2, 4)
  grid.save(f"./out.png")

In [ ]:
send_requests()

In [ ]:
from IPython.display import Image, display
display(Image(filename='out.png'))